In [21]:
from ultralytics import YOLO
import os
from PIL import Image
import cv2 
import matplotlib.pyplot as plt
import numpy as np
global_resize_image=(256,256)

In [10]:
import os
import shutil

def copy_images(image_list_file, source_folder, destination_folder, num_images=100):
    # Create destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Read image names from the text file
    with open(image_list_file, 'r') as file:
        image_names = file.read().splitlines()

    # Take only the first num_images
    image_names = image_names[:num_images]

    # Copy images to destination folder
    for image_name in image_names:
        source_path = os.path.join(source_folder, image_name)
        if os.path.exists(source_path):
            destination_path = os.path.join(destination_folder, image_name)
            shutil.copyfile(source_path, destination_path)
            print(f"Copied {image_name} to {destination_folder}")
        else:
            print(f"Image {image_name} not found in {source_folder}")

# Paths
image_list_file = 'train_split.txt'
source_folder = 'VG_100k'
destination_folder = 'VG_100'

# Copy images
copy_images(image_list_file, source_folder, destination_folder)


Copied 2406949.jpg to VG_100
Copied 2387215.jpg to VG_100
Copied 2372347.jpg to VG_100
Copied 2369202.jpg to VG_100
Copied 2368287.jpg to VG_100
Copied 2352527.jpg to VG_100
Copied 2358049.jpg to VG_100
Copied 2380206.jpg to VG_100
Copied 2375670.jpg to VG_100
Copied 2389065.jpg to VG_100
Copied 2358054.jpg to VG_100
Copied 2389824.jpg to VG_100
Copied 2403652.jpg to VG_100
Copied 2383990.jpg to VG_100
Copied 2389326.jpg to VG_100
Copied 2397541.jpg to VG_100
Copied 2391306.jpg to VG_100
Copied 2403265.jpg to VG_100
Copied 2322521.jpg to VG_100
Copied 2349382.jpg to VG_100
Copied 2409025.jpg to VG_100
Copied 2361341.jpg to VG_100
Copied 2411159.jpg to VG_100
Copied 2372658.jpg to VG_100
Copied 2350124.jpg to VG_100
Copied 2331849.jpg to VG_100
Copied 2405881.jpg to VG_100
Copied 2362382.jpg to VG_100
Copied 2381565.jpg to VG_100
Copied 2408056.jpg to VG_100
Copied 2382766.jpg to VG_100
Copied 2397629.jpg to VG_100
Copied 2406837.jpg to VG_100
Copied 2411655.jpg to VG_100
Copied 2316759

In [19]:
def extract_descriptions(image_list_file, description_file, output_file, num_images):
    # Read image names without extensions from image list file
    with open(image_list_file, 'r') as file:
        image_names = [name.split('.')[0] for name in file.read().splitlines()]

    # Take only the first num_images
    image_names = image_names[:num_images]

    # Read descriptions from train description file and write matching descriptions to output file
    with open(description_file, 'r') as file, open(output_file, 'w') as output:
        for line in file:
            parts = line.strip().split(' ', 1)  # Split line into image name and description
            if parts[0] in image_names:
                output.write(line)

def extract_first_100_image_names(image_list_file, output_file):
    # Read the first 100 image names from image list file
    with open(image_list_file, 'r') as file:
        image_names = file.read().splitlines()[:100]

    # Save the first 100 image names to a new file
    with open(output_file, 'w') as output:
        for name in image_names:
            output.write(name + '\n')

# Paths
image_list_file = 'train_split.txt'
output_file_names = 'first_100_image_names.txt'
description_file = 'train_descriptions.txt'
output_file = '100_train_descriptions.txt'

# Extract descriptions
#extract_descriptions(image_list_file, description_file, output_file,100)


In [20]:
# Extract 100 image_name
extract_first_100_image_names(image_list_file, output_file_names)

In [22]:
#Object Detection using YOLOv8
def detect_objects(imagePath):
    image=cv2.imread(imagePath)
    image=cv2.resize(image,global_resize_image)
    model = YOLO('yolov8x.pt')
    result=model(image)
    return result 

In [25]:
# Region Proposal       
def roi_propose(boxes):
    # Function to calculate IoU
    def calculate_iou(box1, box2):
        # Calculate intersection rectangle coordinates
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        # Calculate area of intersection rectangle
        intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
        
        # Calculate area of both bounding boxes
        box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
        box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
        
        # Calculate IoU
        iou = intersection_area / float(box1_area + box2_area - intersection_area)
        
        return iou

    # Calculate IoU matrix
    num_boxes = len(boxes)
    iou_matrix = np.zeros((num_boxes, num_boxes))
    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            iou_matrix[i, j] = calculate_iou(boxes[i], boxes[j])
            iou_matrix[j, i] = iou_matrix[i, j]  # IoU is symmetric

    # Define threshold for IoU
    iou_threshold = 0.5

    # Merge bounding boxes based on IoU
    merged = set()
    groups = []
    for i in range(num_boxes):
        if i not in merged:
            group = [i]
            for j in range(i + 1, num_boxes):
                if j not in merged and iou_matrix[i, j] >= iou_threshold:
                    group.append(j)
                    merged.add(j)
            groups.append(group)
    # Merge regions completely inside others
    for i, box1 in enumerate(boxes):
        for j, box2 in enumerate(boxes):
            if i != j and calculate_iou(box1, box2) == 1:
                if i in merged:
                    groups.remove(groups[i])
                elif j in merged:
                    groups.remove(groups[j])
    # Create ROIs based on merged bounding boxes
    rois = {}
    for idx, group in enumerate(groups):
        roi_boxes = [boxes[i] for i in group]
        min_x = min(box[0] for box in roi_boxes)
        min_y = min(box[1] for box in roi_boxes)
        max_x = max(box[2] for box in roi_boxes)
        max_y = max(box[3] for box in roi_boxes)
        rois[idx] = (min_x, min_y, max_x, max_y)

    # Function to check if two ROIs overlap
    def is_inside(roi1, roi2):
        x1_min, y1_min, x1_max, y1_max = roi1
        x2_min, y2_min, x2_max, y2_max = roi2
        return (x1_min >= x2_min and y1_min >= y2_min and
                x1_max <= x2_max and y1_max <= y2_max)

    # Function to merge ROIs if one ROI completely encompasses the other
    def merge_overlapping_rois(rois):
        merged_rois = {}
        for idx, roi in rois.items():
            merge = True
            for merged_idx, merged_roi in merged_rois.items():
                if is_inside(roi, merged_roi):
                    # ROI is completely inside another ROI, don't merge
                    merge = False
                    break
                elif is_inside(merged_roi, roi):
                    # Another ROI is completely inside this ROI, merge
                    merged_rois[merged_idx] = roi
                    merge = False
                    break
            if merge:
                merged_rois[idx] = roi
        return merged_rois
    # Merge overlapping ROIs
    merged_rois = merge_overlapping_rois(rois)

    # Print merged ROIs
    for idx, roi in enumerate(merged_rois):
        print(f"ROI {idx}: {roi}")
    '''    
    # Print proposed ROIs
    for roi_id, roi_coords in rois.items():
        print(f"ROI {roi_id}: {roi_coords}")'''
    return merged_rois    


In [29]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to visualize ROIs on the image
def visualize_rois(imagePath, rois, save_path):
    # Read the image
    image=cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,global_resize_image)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Draw ROIs on the image
    for roi_id, roi_coords in rois.items():
        min_x, min_y, max_x, max_y = roi_coords
        width = max_x - min_x
        height = max_y - min_y
        rect = patches.Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Add text label
        ax.text(min_x, min_y - 5, f'ROI {roi_id}', color='r')
     
    image_file = os.path.basename(imagePath) 
    # Remove axis ticks
    ax.axis('off')
    # Save the image with the same name in the output folder
    output_path = os.path.join(save_path, image_file)
    #cv2.imwrite(output_path, img)
        
    print(f"Saved {image_file} to {output_path}")
    # Save the image
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)

    # Show the image
    #plt.show()

# Define image path and save path
#image_path = 'your_image.jpg'
#save_path = 'image_with_rois.jpg'

# Visualize ROIs on the image and save



In [30]:
import os
import cv2

def read_images_and_save(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of all files in the input folder
    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
    
    # Iterate through each image file
    for image_file in image_files:
        # Read the image
        imagePath = os.path.join(input_folder, image_file)
        #img = cv2.imread(imagePath)
        results=detect_objects(imagePath)
        visualize_rois(imagePath, roi_propose(results[0].boxes.xyxy.tolist()), output_folder)

# Example usage:
input_folder = "VG_100"
output_folder = "RoiImages"
read_images_and_save(input_folder, output_folder)



0: 640x640 1 wine glass, 1 cup, 2 sandwichs, 1 dining table, 76.8ms
Speed: 4.5ms preprocess, 76.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
ROI 0: 0
ROI 1: 1
ROI 2: 2
ROI 3: 4
Saved 2315586.jpg to RoiImages\2315586.jpg

0: 640x640 1 person, 1 handbag, 1 potted plant, 1 sink, 1 vase, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
ROI 0: 0
ROI 1: 1
ROI 2: 2
ROI 3: 3
ROI 4: 4
Saved 2316759.jpg to RoiImages\2316759.jpg

0: 640x640 1 refrigerator, 11.5ms
Speed: 2.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
ROI 0: 0
Saved 2317630.jpg to RoiImages\2317630.jpg

0: 640x640 1 clock, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
ROI 0: 0
Saved 2318119.jpg to RoiImages\2318119.jpg

0: 640x640 10 persons, 3 horses, 10.5ms
Speed: 1.5ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

In [32]:
from shutil import copyfile
from collections import Counter

# Function to filter images and save those with more than 2 objects detected
def filter_and_save_images(input_folder, image_names_file, output_folder, max_images=2000):
     # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    image_count = 0
    with open(image_names_file, 'r') as file:
        image_names = file.readlines()
        for name in image_names:
            filename = name.strip()  # Remove newline characters
            imagePath = os.path.join(input_folder, filename)
            if os.path.exists(imagePath):
                objects_detected = detect_objects(imagePath)
                if len(objects_detected[0].boxes) >= 2:
                    # Save the image to the output folder
                    output_path = os.path.join(output_folder, filename)
                    copyfile(imagePath, output_path)
                    image_count += 1
                    if image_count >= max_images:
                        break
            else:
                print(f"Image '{filename}' not found in the input folder.")
    print(f"{image_count} images saved.")

# Example usage:
input_folder = 'VG_100K'
image_names_file = 'train_split.txt'
output_folder = 'filtered_VG_2K'
filter_and_save_images(input_folder, image_names_file, output_folder)



0: 640x640 1 person, 1 bus, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 5 cows, 48.5ms
Speed: 2.9ms preprocess, 48.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 toilet, 1 sink, 48.0ms
Speed: 3.0ms preprocess, 48.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 giraffes, 51.0ms
Speed: 2.0ms preprocess, 51.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 train, 51.0ms
Speed: 2.0ms preprocess, 51.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1 bicycle, 1 car, 1 truck, 3 benchs, 1 laptop, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 elephants, 50.5ms
Speed: 2.0ms preprocess, 50.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 sports ball, 1 ten

In [38]:
import os

def match_descriptions_with_filenames(image_folder, description_file, output_file):
    # Create a set to store the filenames without extensions
    filenames = set()
    for filename in os.listdir(image_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filenames.add(os.path.splitext(filename)[0])

    # Read descriptions from the description file and filter the matched ones
    matched_descriptions = []
    with open(description_file, 'r') as file:
        for line in file:
            parts = line.strip().split(' ', 1)
            if len(parts) >= 2:
                image_name = parts[0]
                if image_name in filenames:
                    matched_descriptions.append(line.strip())

    # Write the matched descriptions to the output file
    with open(output_file, 'w') as outfile:
        for description in matched_descriptions:
            outfile.write(description + '\n')

# Example usage:
image_folder = 'filtered_test_VG_200'
description_file = 'test_descriptions.txt'
output_file = 'filtered_test_descriptions.txt'
match_descriptions_with_filenames(image_folder, description_file, output_file)


In [37]:
import os
import json
from shutil import copyfile

# Function to filter images and save those with more than 2 objects detected
def filter_and_save_images(input_folder, image_names_file, output_folder, max_images=200):
    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    image_count = 0
    with open(image_names_file, 'r') as json_file:
        image_names = json.load(json_file)
        for item in image_names:
            filename = str(item).strip() + '.jpg'  # Convert to string and append '.jpg' extension
            imagePath = os.path.join(input_folder, filename)
            if os.path.exists(imagePath):
                objects_detected = detect_objects(imagePath)
                if len(objects_detected[0].boxes) >= 2:
                    # Save the image to the output folder
                    output_path = os.path.join(output_folder, filename)
                    copyfile(imagePath, output_path)
                    image_count += 1
                    if image_count >= max_images:
                        break
            else:
                print(f"Image '{filename}' not found in the input folder.")
    print(f"{image_count} images saved.")


input_folder = 'VG_100K'
image_names_file = 'test_split.json'
output_folder = 'filtered_test_VG_200'
filter_and_save_images(input_folder, image_names_file, output_folder)



0: 640x640 2 persons, 2 cups, 2 pizzas, 1 chair, 1 dining table, 2 cell phones, 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 sports ball, 1 tennis racket, 11.5ms
Speed: 2.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 airplane, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 toilet, 1 sink, 10.5ms
Speed: 2.0ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 5 cars, 1 bus, 4 traffic lights, 1 handbag, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 surfboard, 17.5ms
Speed: 1.5ms preprocess, 17.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 stop sign, 18.5ms
Speed: 1.0ms preprocess, 18.5ms inference, 0.0ms pos